In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 41.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=1277077d11ad34027f9b1d8abb2c6ea76952bcb8450b6922db028122c5c79b51
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
import xarray as xr

In [ ]:
import pyspark

from pyspark import SparkContext
from pyspark.sql import SparkSession

SparkContext.setSystemProperty('spark.executor.memory', '8g')
SparkContext.setSystemProperty('spark.driver.memory', '45G')

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName("Python Spark").getOrCreate()

In [ ]:
import urllib.request

path_test = "https://dap.ceda.ac.uk/neodc/esacci/land_surface_temperature/data/MULTISENSOR_IRCDR/L3S/0.01/v2.00/monthly/2020/01/ESACCI-LST-L3S-LST-IRCDR_-0.01deg_1MONTHLY_DAY-20200101000000-fv2.00.nc?download=1"

In [ ]:
with urllib.request.urlretrieve(path_test) as f:
  df = xr.open_dataset(f.read())

AttributeError: ignored

In [ ]:
data = urllib.request.urlretrieve(path_test)
data

('/tmp/tmp2ytacrww', <http.client.HTTPMessage at 0x7feaadbdb950>)

In [ ]:
xr_climat = xr.open_dataarray(data, engine = "netcdf4")

ValueError: ignored

In [ ]:
xr_climat.dims

In [12]:
import pandas as pd

In [13]:
url = "https://raw.githubusercontent.com/jldbc/coffee-quality-database/master/data/arabica_data_cleaned.csv"
raw_arabica = pd.read_csv(url)

In [ ]:
raw_arabica

In [10]:
from pyspark.sql.types import *

# Auxiliar functions
def equivalent_type(f):
    if f == 'datetime64[ns]': return TimestampType()
    elif f == 'int64': return LongType()
    elif f == 'int32': return IntegerType()
    elif f == 'float64': return DoubleType()
    elif f == 'float32': return FloatType()
    else: return StringType()

def define_structure(string, format_type):
    try: typo = equivalent_type(format_type)
    except: typo = StringType()
    return StructField(string, typo)

# Given pandas dataframe, it will return a spark's dataframe.
def pandas_to_spark(pandas_df):
    columns = list(pandas_df.columns)
    types = list(pandas_df.dtypes)
    struct_list = []
    for column, typo in zip(columns, types): 
      struct_list.append(define_structure(column, typo))
    p_schema = StructType(struct_list)
    return spark.createDataFrame(pandas_df, p_schema)

In [14]:
df_arabica = pandas_to_spark(raw_arabica)

In [ ]:
df_arabica.show(10)

In [17]:
tempList = [] #Edit01
for col in df_arabica.columns:
        new_name = col.strip()
        new_name = "".join(new_name.split())
        new_name = new_name.replace('.','_') # EDIT
        tempList.append(new_name) #Edit02
print(tempList) #Just for the sake of it #Edit03

df_arabica = df_arabica.toDF(*tempList) #Edit04

['Unnamed:0', 'Species', 'Owner', 'Country_of_Origin', 'Farm_Name', 'Lot_Number', 'Mill', 'ICO_Number', 'Company', 'Altitude', 'Region', 'Producer', 'Number_of_Bags', 'Bag_Weight', 'In_Country_Partner', 'Harvest_Year', 'Grading_Date', 'Owner_1', 'Variety', 'Processing_Method', 'Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance', 'Uniformity', 'Clean_Cup', 'Sweetness', 'Cupper_Points', 'Total_Cup_Points', 'Moisture', 'Category_One_Defects', 'Quakers', 'Color', 'Category_Two_Defects', 'Expiration', 'Certification_Body', 'Certification_Address', 'Certification_Contact', 'unit_of_measurement', 'altitude_low_meters', 'altitude_high_meters', 'altitude_mean_meters']


In [18]:
df_arabica.filter(df_arabica["Country_of_Origin"]=='Brazil')\
          .select('Farm_name')\
          .distinct()\
          .show(100, 100)

+-----------------------------------+
|                          Farm_name|
+-----------------------------------+
|                  fazenda rio verde|
|                    fazenda do lobo|
|                fazenda grota funda|
|                        sitio claro|
|                        santa alina|
|                   fazenda chamusca|
|                        santa maria|
|                        capoeirinha|
|                  fazenda do sertao|
|                         santa fé 2|
|                    café do paraíso|
|            cachoeira da grama farm|
|             são francisco da serra|
|                     fazenda jericó|
|                        sertao farm|
|                   campo das flores|
|                       olhos d'agua|
|       fazenda serra de três barras|
|                    fazenda pantano|
|              pereira estate coffee|
|                          rio verde|
|                  sitío são geraldo|
|                   fazenda baipendi|
|           

In [7]:
df_farms = spark.createDataFrame(
    [
        ("fazenda rio verde",-21.877600079894428, -45.17833587173798 ),  # create your data here, be consistent in the types.
        ("fazenda do lobo", -20.05845833645814, -45.551377369807916),
        ("fazenda grota funda",-4.498676298642221, -46.01438009152326),
        ("sitio claro",-12.404400112670487, -57.0307320652044),
        ("santa alina",-21.76121880758086, -46.674253002625804),
        ("fazenda chamusca",-21.4532483711391, -45.22708818550831),
        ("santa maria",-16.609194745742165, -46.98365752985455),
        ("capoeirinha",-18.64698614414741, -45.796849953979795),
        ("fazenda do sertao",-22.09933939187727, -45.18968007378277),
        ("santa fé 2",-17.582654936016926, -47.2198752950817),
        ("café do paraíso",-22.094568087843687, -45.155496432162685),
        ("cachoeira da grama farm",-21.76626556295618, -46.702161544954144),
        ("são francisco da serra",-22.629913000779446, -44.601043902591805),
        ("fazenda jericó",-18.676315608302055, -45.70281563093488),
        ("sertao farm",-22.099418916815903, -45.18965861611114),
        ("campo das flores",-20.312258124962906, -43.28548394936318),
        ("olhos d'agua",-18.63831446703355, -46.952789277527444),
        ("fazenda serra de três barras",-19.560780988980817, -46.579303497451384),
        ("fazenda pantano",-18.631997524421426, -46.82473127385452),
        ("pereira estate coffee",-22.112480964214626, -45.15508902245408),
        ("rio verde",-21.940510105475326, -45.176192869009476),
        ("sitío são geraldo",-22.59528143237296, -46.66759241371165),
        ("fazenda baipendi",-21.45008093579659, -46.8357381855336),
        ("água limpa",-21.44645605895183, -46.82635044153258),
        ("fazenda kaquend",-21.435236392716707, -46.83244978479159),
        ("fazenda santo antonio",-21.40862617568116, -46.80560442679252),
        ("fazenda vista alegre",-21.44237673402853, -46.818513026788274),
        ("fazenda recreio",-21.780100162105256, -46.67880344388111),
        ("fazenda capoeirnha",-21.761009557830032, -46.67507912298348),
        ("pantano",-21.444638601462618, -46.81747769417689),
        ("fazenda são sebastião",-21.444863288363614, -46.827235570452125),
        ("santa bárbara",-18.52732198399872, -47.569893903127564),
        ("santa mariana",-23.19046310851501, -50.55948683365515),
        ("sertao",-22.095760723175886, -45.189755175672296),
        ("são rafael_ ra/ras certified",-22.784292492565246, -47.032534461666536),
        ("sitío santa luzia",-22.366016295045306, -46.47371725096838),
        ("fazenda são josé mirante",-22.785019797545694, -47.03177457322967),
        ("cianorte",-22.774138613223663, -47.02968245276757),
        ("juliana",-21.46270024286904, -46.832515362350975),
        ("sitío corrego da olaria/são caetano",-22.745464486282955, -47.0338567362583),
        ("fazenda serra negra",-21., -46.674253002625804),
        ("fazendas klem ltda",-20.27591897959779, -41.876604035124465),
        ("castelhana farm",-18.922960557030457, -47.45831617257145),
        ("leticia farm",-19.807319369835636, -42.215697182335234),
        ("helena",-22.107657064931225, -48.32067672583973),
        ("caxambu",-21.339808722609096, -45.42201735838091),
    ],
    ["Farm_name", "long","lat"]  # add your column names here
)

In [19]:
df_arabica = df_arabica.join(df_farms, on=['Farm_name'], how='left_outer')

In [23]:
df_arabica.filter(df_arabica["Country_of_Origin"]=='Brazil')\
          .select(['Farm_name',"long","lat","Total_Cup_Points", "Aroma","Altitude"])\
          .show(100, 100)

+----------------------------+-------------------+-------------------+----------------+-----+--------------+
|                   Farm_name|               long|                lat|Total_Cup_Points|Aroma|      Altitude|
+----------------------------+-------------------+-------------------+----------------+-----+--------------+
|           fazenda rio verde|-21.877600079894428| -45.17833587173798|            84.0| 7.58|          1260|
|           fazenda rio verde|-21.877600079894428| -45.17833587173798|           83.83| 7.92|          1260|
|             fazenda do lobo| -20.05845833645814|-45.551377369807916|           83.17| 7.75|         1000m|
|         fazenda grota funda| -4.498676298642221| -46.01438009152326|           84.92|  8.0|         1200m|
|                 sitio claro|-12.404400112670487|  -57.0307320652044|           84.92|  8.0|        1000 m|
|                 santa alina| -21.76121880758086|-46.674253002625804|           84.33| 8.17|1200m to 1350m|
|            fazend